In [ ]:
# set backend for vtk/pyvista in jupyter
import pyvista as pv
pv.set_jupyter_backend('static')
from pyvirtualdisplay import Display
import numpy as np
#pv.start_xvfb()

In [ ]:
# cfdtools packages
import cfdtools.meshbase.simple as sm
from cfdtools.vtk import vtkMesh

In [ ]:
ni, nj, nk = 10, 10, 10
cube = sm.Cube(ni, nj, nk)
mesh = cube.export_mesh()
vtkmesh = vtkMesh(mesh)

In [ ]:
with Display():
    vtkmesh.plot()

In [ ]:
def map_to_sphere(x, y, z):
    # cube has side range 0 to 1, map to [-1, 1]
    x1, y1, z1 = 2*x-1, 2*y-1, 2*z-1
    x1q, y1q, z1q = (v**2 for v in (x1, y1, z1))
    x2 = x1 * np.sqrt(1 - (      y1q + z1q) / 2 +       y1q * z1q / 3)
    y2 = y1 * np.sqrt(1 - (x1q    +    z1q) / 2 + x1q    *    z1q / 3)
    z2 = z1 * np.sqrt(1 - (x1q + y1q      ) / 2 + x1q * y1q       / 3)
    return x2, y2, z2

mesh_sph = cube.export_mesh()
mesh_sph.morph(map_to_sphere)

In [ ]:
vtkmesh_sph = vtkMesh(mesh_sph)
with Display():
    vtkmesh_sph.plot()

In [ ]:
cube = sm.Cube(12, 12, 12)
mesh = cube.export_mesh()
vtkmesh = vtkMesh(mesh)
def var_u(xyz):
    dp = 2*np.pi
    return 10+2*np.sin(dp*xyz[:,2])+np.sin(2*dp*xyz[:,0])
def var_p(xyz):
    dp = 2*np.pi
    return 100+np.sin(4*dp*xyz[:,2])*np.sin(2*dp*xyz[:,2])
xyz = vtkmesh.pyvista_grid.cell_centers().points
vtkmesh.pyvista_grid.cell_data['U'] = var_u(xyz)
vtkmesh.pyvista_grid.cell_data['P'] = var_p(xyz)

In [ ]:
with Display():
    vtkmesh.plot(scalars='P')

In [ ]:
vtkslice = vtkmesh.vtk_zconvolution(rms=True)
with Display():
    vtkslice.plot()